In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math

In [5]:
data_id = pd.read_excel("../data/F23_Inventory_Demand.xlsx", sheet_name="Chewy")
data_cf = pd.read_excel("../data/F23_Inventory_Demand.xlsx", sheet_name="Conversion Factor")
data_parent_product_mapping = pd.read_excel("../data/F23_Inventory_Demand.xlsx",sheet_name="parent_product_mapping")

In [31]:
# print(data_id.info())
# print(data_cf.info())
# print(data_material_conversion.info())

In [14]:
df = data_id.copy()
df = df[(df["FiscalDate"] <= dt.datetime(2023,6,1)) & 
        (df["FiscalDate"] >= dt.datetime(2022,6,1)) & 
        (df["REGION"] == "NAT")]
df = df[df["BASEPRD_NBR"].isin(data_parent_product_mapping["material_code_packaging"])]
df["LastFiscalYearEndDate"] = dt.datetime(2022,6,5)
df["day_number"] = (df["FiscalDate"] - df["LastFiscalYearEndDate"]).dt.total_seconds()/(24*60*60)
df["week_number"] = df.apply(lambda x: math.floor(x["day_number"]/7), axis = 1)
df = df.merge(data_parent_product_mapping,how = "left", left_on = "BASEPRD_NBR", right_on = "material_code_packaging")
df = df.merge(
    data_cf[["material_cd","conversion_factor"]], 
    how = "left", 
    left_on = "BASEPRD_NBR", 
    right_on = "material_cd"
)

In [15]:
df_demand = df.copy()
df_demand["demand"] = df_demand["EQC_DLVR_QTY"]*df_demand["conversion_factor"]
df_demand = pd.pivot_table(
    df_demand,index = ["week_number","material_code_processing"],values = "demand",aggfunc = "sum"
).reset_index()
df_demand = df_demand[df_demand["week_number"] >= 0].copy()
df_demand = df_demand.rename(columns={"material_code_processing":"material_cd"})
df_demand.head()

,week_number,material_cd,demand
7,0,4104398000,3.365120e+06
8,0,4127706000,2.010240e+06
9,0,4127707000,4.759599e+06
10,0,4127855000,1.469470e+06
11,0,4129853000,4.135440e+05


In [19]:
df_demand.to_csv("../data/Demand.csv")

In [5]:
# df = data_id.copy()
# df = df[(df["FiscalDate"] <= dt.datetime(2023,5,1)) & 
#         (df["FiscalDate"] >= dt.datetime(2022,6,1)) & 
#         (df["REGION"] == "NAT")]
# df = df[df["BASEPRD_NBR"].isin(data_cf["material_cd"])]
# df["LastFiscalYearEndDate"] = dt.datetime(2022,6,5)
# df["day_number"] = (df["FiscalDate"] - df["LastFiscalYearEndDate"]).dt.total_seconds()/(24*60*60)
# df["week_number"] = df.apply(lambda x: math.floor(x["day_number"]/7), axis = 1)
# df_demand = pd.pivot_table(
#     df,index = ["week_number","BASEPRD_NBR"],values = "EQC_DLVR_QTY",aggfunc = "sum"
# ).reset_index().rename(columns = {"EQC_DLVR_QTY":"demand_eqc"})
# df_demand = df_demand.merge(
#     data_cf[["material_cd","conversion_factor"]], 
#     how = "left", 
#     left_on = "BASEPRD_NBR", 
#     right_on = "material_cd"
# )
# df_demand["demand"] = df_demand["demand_eqc"]*df_demand["conversion_factor"]
# df_demand = df_demand[df_demand["week_number"] >= 0].copy()
# df_demand.head()

In [16]:
df_inv = df.copy()
df_inv["inventory"] = df_inv["Inventory"]*df_inv["conversion_factor"]
df_inv = pd.pivot_table(df_inv,index = ["week_number","material_code_processing","FiscalDate"],values="inventory", aggfunc="sum").reset_index()
df_inv_week_end = pd.pivot_table(df,index = ["week_number","material_code_processing"],values = "FiscalDate",aggfunc = "max").reset_index()
df_inv = df_inv_week_end.merge(
    df_inv[["FiscalDate","material_code_processing","inventory"]],
    how = "left",
    on = ["FiscalDate","material_code_processing"]
)
df_inv = df_inv[df_inv["week_number"] >= 0].copy()
df_inv = df_inv.rename(columns = {"material_code_processing":"material_cd"})
df_inv.head()

,week_number,material_cd,FiscalDate,inventory
7,0,4104398000,2022-06-11,2.555394e+07
8,0,4127706000,2022-06-11,2.443675e+07
9,0,4127707000,2022-06-11,2.760308e+07
10,0,4127855000,2022-06-11,1.107513e+07
11,0,4129853000,2022-06-11,4.998072e+06


In [20]:
df_inv.to_csv("../data/inventory.csv")

In [13]:
# df_inv = pd.pivot_table(df,index = ["week_number","BASEPRD_NBR"],values = "FiscalDate",aggfunc = "max").reset_index()
# df_inv = df_inv.merge(
#     df[["FiscalDate","BASEPRD_NBR","Inventory"]],
#     how = "left",
#     on = ["FiscalDate","BASEPRD_NBR"]
# ).rename(columns = {"Inventory":"inventory_eqc"})
# df_inv = df_inv.merge(
#     data_cf[["material_cd","conversion_factor"]],
#     how = "left",
#     left_on = "BASEPRD_NBR",
#     right_on = "material_cd"
# )
# df_inv["inventory"] = df_inv["inventory_eqc"]*df_inv["conversion_factor"]
# df_inv = df_inv[df_inv["week_number"] >= 0].copy()
# df_inv.head()

In [21]:
df = data_id.copy()
df = df[(df["FiscalDate"] < dt.datetime(2022,6,6)) & 
        (df["REGION"] == "NAT")]
df = df[df["BASEPRD_NBR"].isin(data_cf["material_cd"])]
df["NextFiscalYearStartDate"] = dt.datetime(2022,6,6)
df["days_before"] = (df["NextFiscalYearStartDate"] - df["FiscalDate"]).dt.total_seconds()/(24*60*60)
df = df.merge(data_parent_product_mapping,how = "left", left_on = "BASEPRD_NBR", right_on = "material_code_packaging")
df = df.merge(
    data_cf[["material_cd","conversion_factor"]], 
    how = "left", 
    left_on = "BASEPRD_NBR", 
    right_on = "material_cd"
)
df["inventory"] = df["Inventory"]*df["conversion_factor"]
df = pd.pivot_table(df, index = ["material_code_processing","days_before"],values = "inventory",aggfunc="sum").reset_index()
df_inv = pd.pivot_table(df,index = "material_code_processing", values = "days_before",aggfunc="min").reset_index()
df_inv = df_inv.merge(df,how = "left",on = ["material_code_processing","days_before"])
df_inv = df_inv.rename(columns = {"material_code_processing":"material_cd"})
df_inv

,material_cd,days_before,inventory
0,4104398000,1.0,2.665031e+07
1,4127706000,1.0,1.583866e+07
2,4127707000,1.0,2.027423e+07
3,4127855000,1.0,8.462518e+06
4,4129853000,1.0,3.312888e+06
5,4145723000,1.0,1.927490e+07
6,4145796000,1.0,3.454704e+06


In [22]:
df_inv.to_csv("../data/Beginning_Inventory.csv")

In [25]:
# df_demand.to_csv("Demand.csv", index = False)
# df_inv.to_csv("Inventory.csv", index = False)

In [3]:
df_act_prod = pd.read_excel("../data/F23_Prod_Weekly.xlsx", sheet_name="Actual Production Weekly")
df_act_prod = df_act_prod[df_act_prod["Material"].isin(data_parent_product_mapping["material_code_packaging"])]
# df_act_prod = df_act_prod[df_act_prod["Plant"] != "AL"]
df_act_prod = df_act_prod.merge(
    data_cf[["material_cd","conversion_factor"]],
    how = "left", left_on = "Material", right_on = "material_cd")
df_act_prod = df_act_prod.merge(
    data_parent_product_mapping,
    how = "left",left_on = "Material",right_on = "material_code_packaging")
df_act_prod["week_nbr"] = df_act_prod["Fiscal Week ID"] - 1
df_act_prod["actual_production"] = df_act_prod["Actual Production (EQC)"]*df_act_prod["conversion_factor"]
df_act_prod = pd.pivot_table(
    df_act_prod, index = ["material_code_processing","week_nbr"], 
    values="actual_production", aggfunc = "sum").reset_index()
df_act_prod.rename(columns = {"material_code_processing":"material_cd"})
df_act_prod.head()

,material_code_processing,week_nbr,actual_production
0,4104398000,0,7.520716e+06
1,4104398000,1,1.814407e+04
2,4104398000,2,0.000000e+00
3,4104398000,3,0.000000e+00
4,4104398000,4,4.794845e+06


In [4]:
# df_act_prod.to_csv("../Others/Actual_Production.csv")

In [1]:
import polars as pl
import numpy as np
import pandas as pd

In [6]:
df_mps = pl.read_excel(source = "../output/output_20231024214820.xlsx",sheet_name = "MPS")
df_beg_inv = pl.read_excel(source = "../output/output_20231024214820.xlsx",sheet_name = "beginning_inventory")

In [3]:
display(df_mps.head(2))
display(df_beg_inv.head(2))

,material_cd,week_nbr,production_plan,actual_production_non_AL,actual_production_AL,actual_production,demand,total_production_plan,inventory_excess_plan,inventory_shortage_plan,inventory_overflow_plan,product_name,week_start_date,safety_stock,inventory_plan,FiscalDate,inventory_actual,inventory_shortage_actual
i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,str,f64,f64
0,4104398000,0,0,7.5207e6,0.0,7.5207e6,3.3651e6,7.5207e6,2.2296e7,0.0,6.9607e6,"""CH TRAIL MIX F…","""2022-06-06 00:…",8.5100e6,3.7767e7,"""2022-06-11 00:…",2.5554e7,1.7044e7
1,4125119120,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""CH REESES TREA…","""2022-06-06 00:…",0.0,0.0,null,0.0,0.0


material_cd,beginning_inventory
i64,f64
4104398000,2.6650e7
4127706000,1.5839e7


In [7]:
material_cd_list = []
week_nbr_list = []
beginning_inv_list = []
closing_inv_list = []
for material_cd in df_mps["material_cd"].unique():
    for week_nbr in range(52):
        mps_dict = df_mps.filter(
            (pl.col("material_cd") == material_cd) & 
            (pl.col("week_nbr") == week_nbr)
        ).rows(named=True)[0]
        production_actual = mps_dict.get("production_actual")
        demand = mps_dict.get("demand")
        if week_nbr == 0:
            beginning_inv = df_beg_inv.filter(
                pl.col("material_cd") == material_cd
            ).rows(named=True)[0].get("beginning_inventory")
        else:
            beginning_inv = closing_inv
        closing_inv = beginning_inv + production_actual - demand
        material_cd_list.append(material_cd)
        week_nbr_list.append(week_nbr)
        beginning_inv_list.append(beginning_inv)
        closing_inv_list.append(closing_inv)
df = pl.DataFrame(pd.DataFrame({
        "material_cd":material_cd_list,
        "week_nbr":week_nbr_list,
        "beginning_inventory_actual_calculated":beginning_inv_list,
        "closing_inventory_actual_calculated":closing_inv_list
    })
)
df_mps = df_mps.join(df,on = ["material_cd","week_nbr"],how = "left")
df_mps.head()

,material_cd,week_nbr,production_plan,actual_production_non_AL,actual_production_AL,production_actual,demand,total_production_plan,inventory_excess_plan,inventory_shortage_plan,inventory_overflow_plan,product_name,week_start_date,safety_stock,inventory_plan,FiscalDate,inventory_actual,inventory_shortage_actual,beginning_inventory_actual_calculated,closing_inventory_actual_calculated
i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,str,f64,f64,f64,f64
0,4104398000,0,0,7.5207e6,0.0,7.5207e6,3.3651e6,7.5207e6,2.2296e7,0.0,6.9607e6,"""CH TRAIL MIX F…","""2022-06-06 00:…",8.5100e6,3.7767e7,"""2022-06-11 00:…",2.5554e7,1.7044e7,2.6650e7,3.0806e7
1,4125119120,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""CH REESES TREA…","""2022-06-06 00:…",0.0,0.0,null,0.0,0.0,0.0,0.0
2,4127706000,0,3059044,72.0,1.3525056e7,1.3525128e7,2.01024e6,3.059116e6,8.0926e6,0.0,0.0,"""CH SNS ALMOND …","""2022-06-06 00:…",8.7949e6,1.6888e7,"""2022-06-11 00:…",2.4437e7,1.5642e7,1.5839e7,2.7354e7
3,4127707000,0,0,2.4225e6,0.0,2.4225e6,4.7596e6,2.4225e6,4.3979e6,0.0,0.0,"""CH SNS PEANUT …","""2022-06-06 00:…",1.3539e7,1.7937e7,"""2022-06-11 00:…",2.7603e7,1.4064e7,2.0274e7,1.7937e7
4,4127855000,0,0,5.006088e6,0.0,5.006088e6,1.4695e6,5.006088e6,5.1251e6,0.0,0.0,"""CH SNS DK CHOC…","""2022-06-06 00:…",6.8740e6,1.1999e7,"""2022-06-11 00:…",1.1075e7,4.2011e6,8.4625e6,1.1999e7


In [9]:
df_mps.write_excel("../data/mps.xlsx")